In [95]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [96]:
def loadDataframe(soup, df):
    doctor_dict = {}
    doctor_address_keys = ['distance', 'street_and_house_no', 'zipcode_and_city', 'telephone']
    for doctors in soup.find_all('div', {'class':'card dl mb-3'}):
        for i, doctor_data in enumerate(doctors.find_all('div', {'class':'col-sm-4 py-3'})):
            # print(i, '\n', doctor_data.get_text().strip())
            if(i == 0):
                if doctor_data.find('a'):
                    doctor_dict['name'] = doctor_data.find('a').get_text().strip()
                    print(doctor_dict['name'])
                if doctor_data.find('span'):
                    doctor_dict['expertise'] = doctor_data.find('span').get_text().strip()
                
            elif(i == 1):
                if doctor_data:
                    doctor_address_info = doctor_data.get_text().strip()
                    splited = doctor_address_info.split('\n')
                    #print(splited)
                    doctor_address_split = []
                    for el in splited:
                        if el.strip('\r'):
                            doctor_address_split.append(el.strip())
                    doctor_address_dict = dict(zip(doctor_address_keys, doctor_address_split))
                    doctor_dict.update(doctor_address_dict)
                    print(doctor_address_dict)
        doctor_df = pd.DataFrame(doctor_dict, index=[0])
        df = pd.concat([df, doctor_df], ignore_index=True)

        doctor_dict = {}
    
    return(df)

In [97]:
# https://www.arzt-auskunft.de/arzt-auskunft/suche_sn/index.js?a=DL&Mt=&Ft=Allgemeinmedizin&Ft_e=CatId1%3A%3AAllgemeinmedizin%3A%3A2%3B&Ftg=M%C3%BCnchen&Ftg_e=CatId9%3A%3AM%C3%BCnchen%3A%3A48.1471264997798%3A%3A11.552018558777%3B&Otn1=&Otn2=&Sl=&Sd=&St=&Ic1=&Lng=&Db=1
# https://www.arzt-auskunft.de/arzt-auskunft/suche_sn/index.js?a=DL&Ft=Allgemeinmedizin&Ft_e=CatId1%3A%3AAllgemeinmedizin%3A%3A2%3B&Ftg=81377

specializations = [
    'Allgemeinmedizin'
]

locations = [
    '80331', '80333', '80335', '80336', '80337', '80339', '80469', '80538', '80539', '80634', '80636', '80637', '80638', '80639', '80686', '80687', '80689', '80796',
    '80797', '80798', '80799', '80801', '80802', '80803', '80804', '80805', '80807', '80809', '80933', '80935', '80937', '80939', '80992', '80993', '80995', '80997',
    '80999', '81241', '81243', '81245', '81247', '81248', '81249', '81369', '81371', '81373', '81375', '81377', '81379', '81475', '81476', '81477', '81479', '81539',
    '81541', '81543', '81545', '81547', '81549', '81667', '81669', '81671', '81673', '81675', '81677', '81679', '81735', '81737', '81739', '81825', '81827', '81829',
    '81925', '81927', '81929', '82008', '82024', '82031', '82041', '82049', '82054', '82057', '82061', '82064', '82065', '82067', '82069', '82110', '82131', '82140',
    '82152', '82166', '82178', '82194', '82205', '82211', '82216', '82223', '82229', '82234', '82237', '82239', '82256', '82266', '82269', '82272', '82275', '82276',
    '82278', '82279', '82281', '82284'
]

df = pd.DataFrame()

for spl in tqdm(specializations): # desc=f"Running specialization: {spl}"):
    for location in tqdm(locations): # desc=f"Running location: {location}"):
        payload = {'a':'DL', 'Ft':spl, 'Ft_e':f'CatId1%3A%3A{spl}%3A%3A2%3B', 'Ftg':location}
        r = requests.get('https://www.arzt-auskunft.de/arzt-auskunft/suche_sn/index.js', params=payload)
        if(r.status_code == 200):
            soup = BeautifulSoup(r.text, 'html.parser')
            # print(soup)
            df = loadDataframe(soup, df)
        else:
            print('Something went wrong, please check!')


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/112 [00:00<?, ?it/s]

Frau Dr. med. Christa Hutterer
{'distance': '0.1 km', 'street_and_house_no': 'Kaufingerstraße 15', 'zipcode_and_city': '80331 München-Altstadt-Lehel', 'telephone': '089 / 2 00 01 43 - 50'}
Herr Dr. med. Josef J. Dohrenbusch
{'distance': '0.1 km', 'street_and_house_no': 'Kaufingerstraße 12', 'zipcode_and_city': '80331 München', 'telephone': '089 / 24 20 93 98'}
Frau Dr. med. Heidi Herrmann
{'distance': '0.1 km', 'street_and_house_no': 'Altheimer Eck 2', 'zipcode_and_city': '80331 München', 'telephone': '089 / 45 22 81 81'}
Herr Dr. med. (univ.) Thomas Wendel
{'distance': '0.1 km', 'street_and_house_no': 'Altheimer Eck 10', 'zipcode_and_city': '80331 München', 'telephone': '089 / 89 67 40 20'}
Frau Dr. med. Sabine Konz
{'distance': '0.2 km', 'street_and_house_no': 'Eisenmannstraße 4', 'zipcode_and_city': '80331 München', 'telephone': '089 / 37 02 97 67'}
Herr Dr. med. (univ.) Martin Ehrlinger
{'distance': '0.2 km', 'street_and_house_no': 'Eisenmannstraße 4', 'zipcode_and_city': '80331 Mü

In [1]:
print('The lenght of the DataFrame is: ', len(df))
# df.iloc[500:550, :]

NameError: name 'df' is not defined

In [111]:
df.drop_duplicates(inplace=True, ignore_index=True);
print('The length of the dataframe after dropping duplicates is:', len(df))
df.to_csv('./doctors_allgemeine.csv')

The length of the dataframe after dropping duplicates is: 1620
